In [1]:
import pandas as pd
import numpy as np
import os
import re
base_url="https://boardgamegeek.com"
game_info_df=pd.read_csv("data/boardgames2_06022021.csv")
ranking_df=pd.read_csv("data/2021-05-29_game_id_rankings.csv")

In [2]:
# get the top 200 rankings
ranking_df.drop_duplicates(subset=['BoardGameRank'], inplace=True)
ranking_df.set_index('BoardGameRank', inplace=True)
ranking_df.fillna(0, inplace=True)
rank_int_df=ranking_df.astype('int64')
ranking_200_df=rank_int_df.head(200).copy()
ranking_200_dict=ranking_200_df.to_dict()
ranking_200_json=ranking_200_df.to_json('data/ranking.json')
# ranking_200_dict

In [3]:
# check if any game in the top 200 list is not the 20k game info list
game_id_list=game_info_df['objectid']
game_20k_list=[*game_id_list]
game_20k_set=set(game_20k_list)

game_ids=[]
for i in range(200):
    game_id=ranking_200_df.iloc[i].unique()
    game_ids.extend(game_id)
unique_game_ids=set(game_ids)
unique_game_ids.remove(0)

game_out=0
for game_id in unique_game_ids:
    if game_id not in game_20k_set:
        print(f'{game_id} not found')
        game_out+=1
print(f'there are/is {game_out} game(s) from top 200 games that not cover in the 20k game info')

there are/is 0 game(s) from top 200 games that not cover in the 20k game info


In [4]:
# add a column "is_top200" to game_info_df
is_top200_list=[]
for game_id in game_info_df['objectid']:
    if game_id in unique_game_ids:
        is_top200_list.append(True)
    else:
        is_top200_list.append(False)
game_info_df['is_top200']=is_top200_list

In [5]:
# convert unicode to printable format
a=game_info_df['name']
kk=[]
for b in a:
    b=re.sub('\s\s+', ' ', b)
    try:
        c=(b.encode('utf-8').decode('unicode-escape'))
        kk.append(c)
    except:
        x=b.replace('\\u','/u').replace('\\', "").replace('/u','\\u')
        c=(x.encode('utf-8').decode('unicode-escape'))
        kk.append(c)
game_info_df['game_name']=kk

<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\/'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\W'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\T'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\B'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\A'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\O'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequence '\D'
  c=(b.encode('utf-8').decode('unicode-escape'))
<ipython-input-5-996642ce1921>:6: DeprecationWarning: invalid escape sequenc

In [6]:
bg_cat=game_info_df['boardgamecategory']
game_info_df['boardgamecategory']=[cc[1:-1].replace("'",'').split(",") for cc in bg_cat]
bg_pub=game_info_df['boardgamepublisher']
game_info_df['boardgamepublisher']=[cc[1:-1].replace("'",'').split(",") for cc in bg_pub]
bg_mec=game_info_df['boardgamemechanic']
game_info_df['boardgamemechanic']=[cc[1:-1].replace("'",'').split(",") for cc in bg_mec]

In [7]:
game_info_selected_df=game_info_df[['objectid', 'game_name', 'description', 'yearpublished','is_top200',
                                    'average','numplays','maxplaytime','minage', 'languagedependence',
                                    'minplayers','maxplayers', 'minplaytime',  
                                    'news', 'blogs', 'weblink','podcast', 
                                    'boardgamepublisher', 'boardgamecategory', 'boardgamemechanic','gamelink']].copy()
game_info_selected_df.drop_duplicates(subset=['objectid'], inplace=True)
game_info_selected_df=game_info_selected_df.sort_values(by=['game_name']).reset_index(drop=True)
game_info_json=game_info_selected_df.T.to_json('data/game_info.json')
game_info_dict=game_info_selected_df.T.to_dict()
# game_info_dict